# 1. Reading in the data

In [130]:
import pandas as pd

In [131]:
df_linec = pd.read_excel('data_lineC_2.xlsx')
df_linec.head()

,SESSION_DATE,ID_EVALUATION_SESSION,ID_RAT,Name,Birthdate,Gender,WEIGHT,TEMPERATURE,ID_SAMPLE,ID_BL_DOTS,ID_BL_APOPO,STATUS_KNOWNPOS,STATUS_BLINDPOS,REWARD,HIT,SniffTime,SniffThreshold,TB
0,2016-11-03,6924,98,Stephen,2014-07-21,M,NaN,30.5,515562,1,1.0,False,False,False,False,0,0,False
1,2016-11-03,6924,98,Stephen,2014-07-21,M,NaN,30.5,515546,1,1.0,False,False,False,False,0,0,False
2,2016-11-03,6924,98,Stephen,2014-07-21,M,NaN,30.5,515422,1,NaN,False,False,False,False,0,0,True
3,2016-11-03,6924,98,Stephen,2014-07-21,M,NaN,30.5,515559,1,1.0,False,False,False,False,0,0,False
4,2016-11-03,6924,98,Stephen,2014-07-21,M,NaN,30.5,515553,1,1.0,False,False,False,False,0,0,False


In [132]:
df_sniff = pd.read_excel('Snifftime data_March15.2019_example_withIndividualSniffs.xlsx')
df_sniff.head()

,SESSION_DATE,ID_EVALUATION_SESSION,RAT_NAME,RUN,HOLE,LEVEL_NAME,SniffTime,tblEVALUATION.SniffThreshold,tblRAT_SESSION.SniffThreshold,HIT,...,Sniff 1,Sniff 2,Sniff 3,Sniff 4,Sniff 5,Sniff 6,Sniff 7,Sniff 8,Cumulative Sniff Time,Match?
0,2019-03-15,14097,Daudi,A,1,-,0,2500,2500,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,True
1,2019-03-15,14097,Daudi,A,2,2+,3831,2500,2500,True,...,3831.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3831,True
2,2019-03-15,14097,Daudi,A,3,-,0,2500,2500,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,True
3,2019-03-15,14097,Daudi,A,4,-,0,2500,2500,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,True
4,2019-03-15,14097,Daudi,A,5,-,0,2500,2500,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,True


In [133]:
df_key = pd.read_excel('TBdetectionRats_DataVariableKey.xlsx')
df_key.head()

,Data Variable,Description,Date Set (Tab),Unnamed: 3
0,SESSION_DATE,Date the evaluatin session was conducted (i.e....,Both,NaN
1,PROGRAM,This dataset includes results from rats workin...,Both,NaN
2,ID_EVALUATION_SESSION,Unique ID for each evaluation session. Rats ty...,Both,NaN
3,ID_SAMPLE,Unique sample ID (note: patients may provide m...,Both,NaN
4,RAT_NAME,Name of the rat evaluating the sample,Rat Session,NaN


# 2. Profiling the rats

In [134]:
import pandas as pd
from typing import Literal

def fill_weights_day_level(
    df: pd.DataFrame,
    name_col: str = "Name",
    date_col: str = "SESSION_DATE",
    weight_col: str = "WEIGHT",
    agg: Literal["mean", "first"] = "mean"
) -> pd.DataFrame:
    """
    Return a copy of df where missing WEIGHT values are filled on a day-level:
      - Leading days (before first known) are filled with first known weight.
      - Trailing days (after last known) are filled with last known weight.
      - Internal missing days are linearly interpolated based on calendar days.
    The original rows are preserved; only WEIGHT is filled where missing.
    """
    df = df.copy()
    # normalize to midnight datetimes so we have proper DatetimeIndex behavior
    df[date_col] = pd.to_datetime(df[date_col]).dt.normalize()

    # choose aggregator for days (mean or first)
    agg_func = "mean" if agg == "mean" else "first"

    filled_mappings = []  # will collect per-name mappings (date -> filled weight)

    for name, g in df.groupby(name_col):
        # daily aggregated known values (only for dates that exist in raw data)
        daily_known = g.groupby(date_col)[weight_col].agg(agg_func)
        # ensure index is DatetimeIndex normalized
        daily_known.index = pd.to_datetime(daily_known.index).normalize()

        start = g[date_col].min()
        end = g[date_col].max()
        full_idx = pd.date_range(start, end, freq="D")

        # series with full calendar index for this rat
        day_series = pd.Series(index=full_idx, dtype=float)
        # assign known values (days that had rows; may be NaN if all rows that day were NaN)
        day_series.loc[daily_known.index] = daily_known.values

        # find first/last actual known (non-NaN) values
        first_valid = day_series.first_valid_index()
        last_valid = day_series.last_valid_index()

        if first_valid is None:
            # No known weights at all for this rat: leave NaNs (nothing to interpolate/propagate)
            # Map back to rows so merge keeps alignment
            mapping = pd.DataFrame({
                date_col: g[date_col].unique(),
                name_col: name,
                "WEIGHT_FILLED": [float("nan")] * g[date_col].nunique()
            })
            filled_mappings.append(mapping)
            continue

        # Fill leading days (before first_known) with first_known, and trailing after last_known with last_known
        # (this only affects outside the known-range)
        day_series.loc[:first_valid] = day_series.loc[first_valid]
        day_series.loc[last_valid:] = day_series.loc[last_valid]

        # Now interpolate only the internal NaNs using time-aware interpolation
        day_series = day_series.interpolate(method="time")

        # Build mapping DataFrame for dates that actually exist in g
        unique_dates = pd.Series(g[date_col].unique()).dt.normalize()
        mapping = pd.DataFrame({date_col: unique_dates})
        mapping[name_col] = name
        # map date -> interpolated day weight
        mapping["WEIGHT_FILLED"] = mapping[date_col].map(day_series)
        filled_mappings.append(mapping)

    # combine mappings and merge back to original
    mapping_all = pd.concat(filled_mappings, ignore_index=True)
    # ensure datatypes align for merge
    mapping_all[date_col] = pd.to_datetime(mapping_all[date_col]).dt.normalize()
    df = df.merge(mapping_all, on=[name_col, date_col], how="left")

    # fill original WEIGHT only where missing
    df[weight_col] = df[weight_col].fillna(df["WEIGHT_FILLED"])
    df = df.drop(columns=["WEIGHT_FILLED"])

    return df


In [135]:
# Make sure SESSION_DATE is datetime and WEIGHT is numeric
df_linec["SESSION_DATE"] = pd.to_datetime(df_linec["SESSION_DATE"])
df_linec["WEIGHT"] = pd.to_numeric(df_linec["WEIGHT"], errors="coerce")

# Apply the function from above
df_linec_filled = fill_weights_day_level(
    df_linec,
    name_col="Name",
    date_col="SESSION_DATE",
    weight_col="WEIGHT",
    agg="first"   # or "mean" if you prefer daily average
)

# Now df_linec_filled has WEIGHT filled on day-level


In [136]:
df_linec_filled.head()

,SESSION_DATE,ID_EVALUATION_SESSION,ID_RAT,Name,Birthdate,Gender,WEIGHT,TEMPERATURE,ID_SAMPLE,ID_BL_DOTS,ID_BL_APOPO,STATUS_KNOWNPOS,STATUS_BLINDPOS,REWARD,HIT,SniffTime,SniffThreshold,TB
0,2016-11-03,6924,98,Stephen,2014-07-21,M,866.428571,30.5,515562,1,1.0,False,False,False,False,0,0,False
1,2016-11-03,6924,98,Stephen,2014-07-21,M,866.428571,30.5,515546,1,1.0,False,False,False,False,0,0,False
2,2016-11-03,6924,98,Stephen,2014-07-21,M,866.428571,30.5,515422,1,NaN,False,False,False,False,0,0,True
3,2016-11-03,6924,98,Stephen,2014-07-21,M,866.428571,30.5,515559,1,1.0,False,False,False,False,0,0,False
4,2016-11-03,6924,98,Stephen,2014-07-21,M,866.428571,30.5,515553,1,1.0,False,False,False,False,0,0,False


In [137]:
# check if any weights are still missing
missing_weights = df_linec_filled['WEIGHT'].isnull().sum()
print(f"Number of missing WEIGHT entries after filling: {missing_weights}")

Number of missing WEIGHT entries after filling: 0


In [138]:
# delete all columns but Birthdate, Gender, WEIGHT and TB
df_linec_filled = df_linec_filled[['Birthdate', 'Gender','TEMPERATURE', 'WEIGHT', 'TB', 'HIT']]
df_linec_filled.head()

,Birthdate,Gender,TEMPERATURE,WEIGHT,TB,HIT
0,2014-07-21,M,30.5,866.428571,False,False
1,2014-07-21,M,30.5,866.428571,False,False
2,2014-07-21,M,30.5,866.428571,True,False
3,2014-07-21,M,30.5,866.428571,False,False
4,2014-07-21,M,30.5,866.428571,False,False
